# Preliminaries

In [ ]:
%pip install --upgrade -q pip cryptography==43.0.3 nvidia-nvshmem-cu12 pandas==2.2.2 numpy==2.0.2 yandex-cloud-ml-sdk nvidia-ml-py3 fastai pydantic==2.12.3 fsspec==2025.3.0 datasets dvc soxr pyopenssl==24.2.1
%pip install --upgrade -q torchaudio torchdata torchtext torch torchvision --index-url https://download.pytorch.org/whl/cu126
%pip install --upgrade -q pybind11 sentencepiece transformers
%pip install --upgrade -q peft dotenv bitsandbytes

In [ ]:
import os
import itertools
import random
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
from transformers.trainer_callback import EarlyStoppingCallback
from peft import LoraConfig, TaskType, get_peft_model

os.environ["HF_TOKEN"] = "YOUR_HF_TOKEN"
# os.environ["HF_HOME"] = os.environ.get("TRANSFORMERS_CACHE", "~/.cache/huggingface")

# Load the data

In [ ]:
df_train = pd.read_parquet("train.parquet")[["ce", "ru"]]
df_dev = pd.read_parquet("dev.parquet")[["ce", "ru"]]

# Download the original model and its tokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "jbochi/madlad400-3b-mt"
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    device_map=None,
)

model = model.to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Dataset configuration

In [ ]:
MAX_LENGTH = 256

In [ ]:
class ParallelSentencesDataset(Dataset):
    def __init__(self, df, tokenizer, main_lang, max_length=MAX_LENGTH):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.main_lang = main_lang

        self.languages = self.df.columns.to_list()
        self.secondary_langs = list(
            filter(lambda x: x != self.main_lang, self.languages)
        )
        self.directions = list(
            itertools.product([self.main_lang], self.secondary_langs)
        ) + list(itertools.product(self.secondary_langs, [self.main_lang]))

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        # select translation direction
        direction = random.choice(self.directions)

        src_lang, tgt_lang = direction

        src_text, tgt_text = self.df.iloc[idx][[src_lang, tgt_lang]]
        tokens = self.tokenizer(
            f"<2{tgt_lang}> {src_text}",
            text_target=tgt_text,
            truncation=True,
            max_length=self.max_length,
        )
        return tokens


dataset_train = ParallelSentencesDataset(
    df=df_train,
    main_lang="ce",
    tokenizer=tokenizer,
)

dataset_eval = ParallelSentencesDataset(
    df=df_dev,
    main_lang="ce",
    tokenizer=tokenizer,
)

# Initialize LoRA adapter

Check the modules of MADLAD-400

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,
    lora_alpha=64,
    lora_dropout=0.1,
    bias="all",
    target_modules=["q", "v", "k", "o", "wi_0", "wi_1", "wo", "lm_head"], # train all linear layers with LoRA
    modules_to_save=[
        "shared", # train embeddings as they are, because no adapter may be applied to them
    ],
    use_rslora=True,  # rank-stabilized LoRA for better stability
    # use_dora=True,
    layers_to_transform=None, # apply LoRA to all the layers of the model
    init_lora_weights="pissa",
)

lora_model = get_peft_model(model, peft_config)

lora_model.print_trainable_parameters()

trainable params: 364,741,632 || all params: 3,305,114,624 || trainable%: 11.0357


# Set up the collator

In [ ]:
# this object is used for proper post processing of tokenized batches before feeding them to the model during train process

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=lora_model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8,
)

# Trainig arguments

In [ ]:
BATCH_SIZE = 10
NUM_EPOCHS = 16

training_args = Seq2SeqTrainingArguments(
    output_dir="./madlad-lora-checkpoints",
    learning_rate=1e-4,
    optim="adamw_torch_fused",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=12,
    # it may be used for saving GPU memory, however we've failed with setting this up
    gradient_checkpointing=False,

    # must have for new languages, as the gradient is tending to be too big at the start
    max_grad_norm=0.5,


    warmup_steps=200,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    num_train_epochs=NUM_EPOCHS,

    # three must have parameters here! without these settings the gradients will very soon blow up!
    fp16=False,
    bf16=True,
    bf16_full_eval=True,

    logging_steps=20,
    save_strategy="steps",
    save_steps=200,
    eval_strategy="steps",
    eval_steps=200,
    save_total_limit=3,
    predict_with_generate=True,
    generation_max_length=MAX_LENGTH,
    dataloader_pin_memory=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    generation_num_beams=4,
    report_to="none",
    logging_dir="./lora_logs",
)

# Train the model

In [ ]:
if training_args.gradient_checkpointing:
    lora_model.gradient_checkpointing_enable()

lora_model.train()

trainer = Seq2SeqTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    data_collator=data_collator,
    # this callback was too harsh, it's better to set it as early_stopping_patience=5, early_stopping_threshold=0.001
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3, 
            early_stopping_threshold=0.01,
        )
    ],
)


trainer.train()

lora_model.save_pretrained("./madlad-lora-final")
tokenizer.save_pretrained("./madlad-lora-final")

  0%|          | 20/28720 [02:34<60:28:37,  7.59s/it]

{'loss': 5.9047, 'grad_norm': 7.833569526672363, 'learning_rate': 9.5e-06, 'epoch': 0.01}


  0%|          | 40/28720 [05:07<60:46:35,  7.63s/it]

{'loss': 5.2804, 'grad_norm': 4.561985015869141, 'learning_rate': 1.9500000000000003e-05, 'epoch': 0.03}


  0%|          | 60/28720 [07:40<60:55:42,  7.65s/it]

...

{'eval_loss': 1.6028015613555908, 'eval_runtime': 7.5206, 'eval_samples_per_second': 47.868, 'eval_steps_per_second': 4.787, 'epoch': 3.34}


/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:279: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
 17%|█▋        | 4800/28720 [10:51:05<50:50:10,  7.65s/it]

{'train_runtime': 39065.3069, 'train_samples_per_second': 88.198, 'train_steps_per_second': 0.735, 'train_loss': 1.906110961039861, 'epoch': 3.34}


 17%|█▋        | 4800/28720 [10:51:05<54:04:35,  8.14s/it]
/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:279: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('./madlad-lora-final/tokenizer_config.json',
 './madlad-lora-final/special_tokens_map.json',
 './madlad-lora-final/spiece.model',
 './madlad-lora-final/added_tokens.json')

# Check if it provides translations

In [12]:
lora_model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): ModulesToSaveWrapper(
        (original_module): Embedding(256000, 1024)
        (modules_to_save): ModuleDict(
          (default): Embedding(256000, 1024)
        )
      )
      (encoder): T5Stack(
        (embed_tokens): Embedding(256000, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                   

In [ ]:
inputs = tokenizer("<2ce> I love Mary, she cooks very well!", return_tensors="pt").to(
    device
)
outputs = lora_model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Суна Марем дукха веза, цо чӀогӀа дика яа йо!']


In [ ]:
inputs = tokenizer(
    "<2en> Стигална кӀел къахьоьгуш, ша мел динчу хӀуманах буьсун болу хӀун пайда оьцу адамо?",
    return_tensors="pt",
).to(device)
outputs = lora_model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['What profit has man, when he tries under the heaven, and when he tries, and']


In [ ]:
inputs = tokenizer(
    "<2ce> Когда я зашел в пещеру, Павел вышел наружу?", return_tensors="pt"
).to(device)
outputs = lora_model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Со хьех чу ваьлча, ПахӀал араваьлла.']
